<a href="https://colab.research.google.com/github/naoya1110/zakkuri_AI_nyumon/blob/main/Introduction_to_Object_Detection_Model_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 物体検出モデル超ざっくり入門

## YOLOv8のセットアップ
物体検出モデルYOLOシリーズの最新版YOLOv8をインストールします。

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

## パッケージのインポート
必要なパッケージをインポートします。

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

## サンプル画像
サンプル画像をダウンロードします。

In [ ]:
!wget https://ultralytics.com/images/zidane.jpg

ダウンロードした画像を表示してみましょう。

In [ ]:
img = cv2.imread("zidane.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

## YOLOv8を使った物体検出


### 学習済みモデルの読み込み
COCOデータセットで学習されたYOLOv8のモデルを読み込みます。YOLOv8にもいくつかの種類があります。詳しくは以下のリンクを参照してください。

https://github.com/ultralytics/ultralytics#models

In [ ]:
# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

### 物体検出
学習済みモデルを使って物体検出（推論）します。

In [ ]:
# Predict on an image
results = model("zidane.jpg", save=True)  # predict on an image

物体検出の結果画像は`runs/detect/predict`に保存されています。これを表示してみましょう。

In [ ]:
img = cv2.imread("runs/detect/predict/zidane.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

### 検出結果の処理
検出結果は`results`に保存されています。この中から必要な情報を抜き出します。

In [ ]:
detections = results[0].boxes.data
detections = detections.to("cpu").numpy()
detections

`detections`は`n`行6列のNumpy配列です。ここで`n`は検出された物体の数です。各行には個々の物体に対するデータが以下のように入っています。

```
[left, top, right, bottom, confidence, class_id]

# left: 左端の座標
# top: 上端の座標
# right: 右端の座標
# bottom: 下端の座標
# conf: 予測されたクラスの信頼度
# class_id: 予測されたクラスのID
```


クラスIDとクラス名の関係は次のようになっています。

In [ ]:
classnames = results[0].names
classnames

`detections`のデータを使って検出結果を描画してみましょう。

In [ ]:
### 検出結果の整理
det = detections[0]    # 1つ目の検出結果を取得
left, top, right, bottom, conf, id = det    # 座標，信頼度, クラスIDを取得
classname = classnames[int(id)]    # クラスIDからクラス名に変換

### 元の画像を読み込み
img = cv2.imread("zidane.jpg")

### 枠(バウンディングボックス)を描画
img = cv2.rectangle(img,
                    pt1 = (int(left), int(top)),
                    pt2 = (int(right), int(bottom)),
                    color = (0,255,0),
                    thickness = 3,
                    lineType = cv2.LINE_AA)

### テキストを描画
cv2.putText(img,
            text = f"{classname} {conf:.2f}",
            org = (int(left), int(top)-15),
            fontFace = cv2.FONT_HERSHEY_SIMPLEX,
            fontScale = 1.5,
            color = (0, 255, 0),
            thickness = 3,
            lineType = cv2.LINE_AA)


img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

### いろいろ検出してみましょう
Webカメラを使って写真を撮ります。

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])

    with open(filename, 'wb') as f:
        f.write(binary)
        return filename

try:
    filename = take_photo()
    print('Saved to {}'.format(filename))

    # Show the image which was just taken.
    display(Image(filename))
    
except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

写真は`photo.jpg`に保存されています。推論してみましょう。

In [ ]:
# 推論
results = model("photo.jpg", save=True)

# 結果の表示
img = cv2.imread("runs/detect/predict/photo.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)